### Exercise - Matrix Multiplication

### Random Matrix Generation

In [66]:
from random import uniform

# MATRIX_A_ROWS = 3
# MATRIX_A_COLS = 3
# MATRIX_B_ROWS = 3
# MATRIX_B_COLS = 3

MATRIX_SIZE_EXPONENT = int(input("Wprowadź liczbę n (rozmiar macierzy == 2^n)"))
ALGORITHM_SWITCH_BOUNDARY_SIZE = int(input("Wprowadź liczbę n po której nastąpi zmiana wykorzystywanego algorytmu"))

# MATRIX_A_ROWS = int(input("Wprowadź liczbę więrszy macierzy A: "))
# MATRIX_A_COLS = int(input("Wprowadź liczbę kolumn macierzy A: "))
# MATRIX_B_ROWS = int(input("Wprowadź liczbę więrszy macierzy B: "))
# MATRIX_B_COLS = int(input("Wprowadź liczbę kolumn macierzy B: "))

def createMatrix(rows, cols):
    matrix = []
    for i in range(rows):
        matrix.append([])
        for j in range(cols):
            matrix[i].append(uniform(0, 10))
    return matrix

def canMultiply():
    if (MATRIX_A_COLS != MATRIX_B_ROWS):
        print("Błędne rozmiary macierzy")
        return False
    return True

def printMatrix(matrix):
    for row in matrix:
        print(row)

generatedMatrixA = createMatrix(2 ** MATRIX_SIZE_EXPONENT, 2** MATRIX_SIZE_EXPONENT)
generatedMatrixB = createMatrix(2 ** MATRIX_SIZE_EXPONENT, 2** MATRIX_SIZE_EXPONENT)

Wprowadź liczbę n (rozmiar macierzy == 2^n)7
Wprowadź liczbę n po której nastąpi zmiana wykorzystywanego algorytmu2


In [15]:
printMatrix(generatedMatrixA)
printMatrix(generatedMatrixB)

[9.562934029998045, 4.201302868782168, 7.162748173928529, 9.037769392784924]
[5.652381433308439, 3.9556803531629003, 4.124046075443352, 2.3271950122063068]
[3.09012015980099, 7.073779363343252, 6.7245042677210565, 3.6491970346499523]
[1.656452091229783, 1.4680752934493269, 6.303112314105878, 9.79215080516269]
[5.81898121735001, 0.39458181389361413, 9.504554154974693, 1.9456181198556954]
[0.049935405715602066, 2.9254922835162356, 8.16954219874251, 3.391417911328504]
[7.459134040332405, 8.160778186945828, 0.9026820422427084, 7.326493784031262]
[0.05670435432961862, 7.986531816645374, 3.863227582600838, 7.93547325130967]


### Basic Naive Algorithm

In [28]:

def basicMultiplication(matrixA, matrixB):
    matrix = []
    for i in range(len(matrixA)):
        matrix.append([])
        for j in range(len(matrixB)):
            sum = 0
            for k in range(len(matrixB)):
                #print(matrixA[i][k]);
                #print(matrixB[k][j]);
                sum += matrixA[i][k] * matrixB[k][j]
            matrix[i].append(sum)
    return matrix

def basicMultiplicationWithOperationCounting(matrixA, matrixB):
    matrix = []
    fpOperations = 0
    for i in range(len(matrixA)):
        matrix.append([])
        for j in range(len(matrixA)):
            sum = 0
            for k in range(len(matrixA)):
                #print(matrixA[i][k]);
                #print(matrixB[k][j]);
                sum += matrixA[i][k] * matrixB[k][j]
                fpOperations += 2
            matrix[i].append(sum)
    return matrix, fpOperations


### Strassen Algorithm

In [42]:
def matrixPairwiseOperation(matrixA, matrixB, op):
    sumMatrix = [[None] * len(matrixA[0]) for x in matrixA]
    for row,_ in enumerate(matrixA):
        for col,_ in enumerate(matrixA[row]):
            sumMatrix[row][col] = op(matrixA[row][col], matrixB[row][col])
    return sumMatrix

def matrixPairwiseOperationWithOperationCounting(matrixA, matrixB, op):
    fpOperations = 0
    sumMatrix = [[None] * len(matrixA[0]) for x in matrixA]
    for row,_ in enumerate(matrixA):
        for col,_ in enumerate(matrixA[row]):
            sumMatrix[row][col] = op(matrixA[row][col], matrixB[row][col])
            fpOperations += 1
    return sumMatrix, fpOperations

def add(matrixA, matrixB):
    return matrixPairwiseOperation(matrixA, matrixB, lambda x,y : x + y)
    
def addWithOperationCounting(matrixA, matrixB):
    return matrixPairwiseOperationWithOperationCounting(matrixA, matrixB, lambda x,y : x + y)

def sub(matrixA, matrixB):
    return matrixPairwiseOperation(matrixA, matrixB, lambda x,y : x - y)
    
def subWithOperationCounting(matrixA, matrixB):
    return matrixPairwiseOperationWithOperationCounting(matrixA, matrixB, lambda x,y : x - y)


In [56]:
def splitMatrixIntoFourParts(matrix):
    length = len(matrix)
    firstHalf = matrix[0:length//2]
    secondHalf = matrix[length//2:]
    firstQuarter = [x[0:length//2] for x in firstHalf]
    secondQuarter = [x[length//2:] for x in firstHalf]
    thirdQuarter = [x[0:length//2] for x in secondHalf]
    fourthQuarter = [x[length//2:] for x in secondHalf]
    return [[firstQuarter, secondQuarter], [thirdQuarter, fourthQuarter]]

def joinMatrixFromFourParts(first, second, third, fourth):
    firstHalf = [x + y for (x,y) in zip(first, second)]
    secondHalf = [x + y for (x,y) in zip(third, fourth)]
    return firstHalf + secondHalf

def strassenAlgorithm(matrixA, matrixB, multiplyMatrix=None): #we pass multiply matrix to enable algorithm to change multiplication strategy during execution
    mul = multiplyMatrix if multiplyMatrix is not None else strassenAlgorithm
    length = len(matrixA)
    
    if length == 1:
        return [[matrixA[0][0] * matrixB[0][0]]]
    
    aParts = splitMatrixIntoFourParts(matrixA)
    bParts = splitMatrixIntoFourParts(matrixB)
    
    P1 = mul(add(aParts[0][0], aParts[1][1]), add(bParts[0][0], bParts[1][1]))
    P2 = mul(add(aParts[1][0], aParts[1][1]), bParts[0][0])
    P3 = mul(aParts[0][0],sub(bParts[0][1], bParts[1][1]))
    P4 = mul(aParts[1][1], sub(bParts[1][0], bParts[0][0]))
    P5 = mul(add(aParts[0][0], aParts[0][1]), bParts[1][1])
    P6 = mul(sub(aParts[1][0], aParts[0][0]), add(bParts[0][0], bParts[0][1]))
    P7 = mul(sub(aParts[0][1], aParts[1][1]), add(bParts[1][0], bParts[1][1]))
    
    C11 = add(sub(add(P1, P4), P5), P7)
    C12 = add(P3, P5)
    C21 = add(P2, P4)
    C22 = add(add(sub(P1, P2), P3), P6)
    
    return joinMatrixFromFourParts(C11, C12, C21, C22)

def strassenAlgorithmWithOperationCounting(matrixA, matrixB, multiplyMatrixWithCounting=None): #we pass multiply matrix to enable algorithm to change multiplication strategy during execution
    mulWithCounting = multiplyMatrixWithCounting if multiplyMatrixWithCounting is not None else strassenAlgorithmWithOperationCounting
    length = len(matrixA)
    
    if length == 1:
        return [[matrixA[0][0] * matrixB[0][0]]], 1
    
    aParts = splitMatrixIntoFourParts(matrixA)
    bParts = splitMatrixIntoFourParts(matrixB)
    
    fpOperations = 0
    def add(x, y):
        res = addWithOperationCounting(x, y)
        nonlocal fpOperations
        fpOperations += res[1]
        return res[0]
    
    def sub(x, y):
        res = subWithOperationCounting(x, y)
        nonlocal fpOperations 
        fpOperations += res[1]
        return res[0]
    
    def mul(x, y):
        res = mulWithCounting(x, y)
        nonlocal fpOperations
        fpOperations += res[1]
        return res[0]
    
    P1 = mul(add(aParts[0][0], aParts[1][1]), add(bParts[0][0], bParts[1][1]))
    P2 = mul(add(aParts[1][0], aParts[1][1]), bParts[0][0])
    P3 = mul(aParts[0][0],sub(bParts[0][1], bParts[1][1]))
    P4 = mul(aParts[1][1], sub(bParts[1][0], bParts[0][0]))
    P5 = mul(add(aParts[0][0], aParts[0][1]), bParts[1][1])
    P6 = mul(sub(aParts[1][0], aParts[0][0]), add(bParts[0][0], bParts[0][1]))
    P7 = mul(sub(aParts[0][1], aParts[1][1]), add(bParts[1][0], bParts[1][1]))
    
    C11 = add(sub(add(P1, P4), P5), P7)
    C12 = add(P3, P5)
    C21 = add(P2, P4)
    C22 = add(add(sub(P1, P2), P3), P6)
    
    
    return joinMatrixFromFourParts(C11, C12, C21, C22), fpOperations
        
        


### Basic test for strassen algorithm utils

In [57]:
printMatrix(generatedMatrixA)
printMatrix(splitMatrixIntoFourParts(generatedMatrixA))
parts = splitMatrixIntoFourParts(generatedMatrixA)
printMatrix(joinMatrixFromFourParts(parts[0][0], parts[0][1], parts[1][0], parts[1][1]))

[9.562934029998045, 4.201302868782168, 7.162748173928529, 9.037769392784924]
[5.652381433308439, 3.9556803531629003, 4.124046075443352, 2.3271950122063068]
[3.09012015980099, 7.073779363343252, 6.7245042677210565, 3.6491970346499523]
[1.656452091229783, 1.4680752934493269, 6.303112314105878, 9.79215080516269]
[[[9.562934029998045, 4.201302868782168], [5.652381433308439, 3.9556803531629003]], [[7.162748173928529, 9.037769392784924], [4.124046075443352, 2.3271950122063068]]]
[[[3.09012015980099, 7.073779363343252], [1.656452091229783, 1.4680752934493269]], [[6.7245042677210565, 3.6491970346499523], [6.303112314105878, 9.79215080516269]]]
[9.562934029998045, 4.201302868782168, 7.162748173928529, 9.037769392784924]
[5.652381433308439, 3.9556803531629003, 4.124046075443352, 2.3271950122063068]
[3.09012015980099, 7.073779363343252, 6.7245042677210565, 3.6491970346499523]
[1.656452091229783, 1.4680752934493269, 6.303112314105878, 9.79215080516269]


### Basic test checking if algorithms answers are the same

In [58]:
printMatrix(basicMultiplication(generatedMatrixA, generatedMatrixB))

[109.79670687107955, 146.69827084240097, 166.59478959788507, 157.05099876867115]
[63.9824044528412, 66.04428153097001, 98.75264930228525, 73.09493264492639]
[68.70048726107782, 105.9352081290425, 107.32752130169531, 108.22747998444171]
[57.28318975279621, 134.59207425503848, 71.25635504665388, 132.086744025259]


In [59]:
printMatrix(strassenAlgorithm(generatedMatrixA, generatedMatrixB))

[109.79670687107946, 146.69827084240097, 166.5947895978851, 157.05099876867115]
[63.982404452841166, 66.04428153097001, 98.75264930228528, 73.09493264492643]
[68.70048726107782, 105.9352081290425, 107.32752130169531, 108.22747998444174]
[57.28318975279622, 134.59207425503848, 71.25635504665388, 132.08674402525904]


In [67]:
testRes = basicMultiplicationWithOperationCounting(generatedMatrixA, generatedMatrixB)
print(testRes[1])
# printMatrix(testRes[0])

4194304


In [68]:
testRes = strassenAlgorithmWithOperationCounting(generatedMatrixA, generatedMatrixB)
print(testRes[1])
# printMatrix(testRes[0])

5666497


In [62]:
testA = createMatrix(2**6, 2**6)
testB = createMatrix(2**6, 2**6)

In [12]:
test = sub(basicMultiplication(generatedMatrixA, generatedMatrixB), strassenAlgorithm(generatedMatrixA, generatedMatrixB))

In [13]:
print(sum([sum(x) for x in test]))

-3.410605131648481e-13


### Hybrid algorithm

In [ ]:
def main():
    if (MATRIX_A_ROWS < 1 or MATRIX_A_COLS < 1 or MATRIX_B_ROWS < 1 or MATRIX_B_COLS < 1):
        return print("Liczba wierszy/kolumn musi być liczbą całkowitą większa od 0")

    matrixA = createMatrix(MATRIX_A_ROWS, MATRIX_A_COLS)
    matrixB = createMatrix(MATRIX_B_ROWS, MATRIX_B_COLS)

    print("Macierz A:")
    printMatrix(matrixA)
    print("Macierz B:")
    printMatrix(matrixA)

    if (canMultiply()):
        if (MATRIX_A_ROWS > 2 and MATRIX_A_COLS > 2):
            print("wynik mnożenia: ")
            printMatrix(basicMultiplication(matrixA,matrixB))
        else:
            StressenAlgorithm()

main()

"""
TODO
mnożenie liczba x macierz
mnożenie metodą Stressena
"""